<a href="https://colab.research.google.com/github/1digitaldesign/sample-code/blob/main/ETL_%26_SQL_Tai%2CBetty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook Setup

* Part 1. ETL

* Part 2. SQL 

## Part 1: ETL


In [1]:
#@title Installation of Necessary Packages 
%%capture
!pip3 install pyspark
!pip3 install hide_code
!pip3 install fancyimpute
!pip3 install miceforest
!pip3 install scikit-learn
%pip install mlxtend --upgrade

On the left, download the dataset [`usa_00001.csv`](https://drive.google.com/file/d/128-eo8QVTYu20aRzA_-7OTxPOjyS-SRp/view?usp=share_link) and on the left hand side, there is a folder to click on called `Files` and then `Upload to Session Storage` this file [`usa_00001.csv`](https://drive.google.com/file/d/128-eo8QVTYu20aRzA_-7OTxPOjyS-SRp/view?usp=share_link). 

In [2]:
#@title Installation of Necessary Libraries 
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
import sqlite3
import requests
import os
import seaborn as sns
import matplotlib.pyplot as plt

## Data Extraction

In [3]:
df = pd.read_csv('/content/usa_00001.csv')

## Data Transformation
Data transformation is an integral step for data analysis and modeling without errors and improve the performance of the data. 

* Cleaning 
* Normalization
* Reduction
* Discretization
* Aggregation






In [4]:
df.isnull().sum()
df.shape

(438666, 32)

In [5]:
for col_name in df.columns: 
    print(col_name)

YEAR
SAMPLE
SERIAL
CBSERIAL
HHWT
CLUSTER
STRATA
GQ
PERNUM
PERWT
SEX
AGE
MARST
BIRTHYR
MARRNO
DIVINYR
WIDINYR
SCHOOL
EDUC
EDUCD
DEGFIELD
DEGFIELDD
DEGFIELD2
DEGFIELD2D
EMPSTAT
EMPSTATD
OCC2010
IND
INDNAICS
WKSWORK1
FTOTINC
INCWAGE


In [6]:
df.describe().style.highlight_max(color="darkred")

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,STRATA,GQ,PERNUM,PERWT,SEX,AGE,MARST,BIRTHYR,MARRNO,DIVINYR,WIDINYR,SCHOOL,EDUC,EDUCD,DEGFIELD,DEGFIELDD,DEGFIELD2,DEGFIELD2D,EMPSTAT,EMPSTATD,OCC2010,IND,WKSWORK1,FTOTINC,INCWAGE
count,438666.000000,438666.000000,438666.000000,438666.000000,438665.000000,438665.000000,438665.000000,438665.000000,438665.000000,438665.000000,438665.000000,438665.000000,438665.000000,438665.000000,438665.000000,438665.000000,438665.000000,438665.000000,438665.000000,438665.000000,438665.000000,438665.000000,438665.000000,438665.000000,438665.000000,438665.000000,438665.000000,438665.000000,438665.000000,438665.000000,438665.000000
mean,2021.000000,202101.000000,98844.701584,2020996517875.833984,97.625849,2021000988446.024414,378802.059227,1.148077,2.173770,100.874323,1.505698,42.023316,3.713948,1978.976684,0.728638,0.566058,0.566683,1.199932,6.226248,64.673393,11.281930,1129.589543,1.138792,114.047952,1.583760,15.877727,6570.088781,3722.867450,21.604495,694552.676872,195088.402198
std,0.000000,0.000000,54100.192938,3051373300.137540,78.732127,541000.938130,316217.332307,0.599769,1.452446,82.121108,0.499968,23.643354,2.301207,23.643354,0.771796,0.506055,0.507206,0.460731,3.319526,33.038127,20.896005,2091.731540,7.437090,744.691227,1.137499,11.359478,3509.028486,3747.726583,24.596056,2328276.360794,365590.015772
min,2021.000000,202101.000000,1.000000,20210010.000000,1.000000,2021000000011.000000,10001.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1927.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,0.000000,0.000000,-6800.000000,0.000000
25%,2021.000000,202101.000000,50172.250000,2021000300395.250000,51.000000,2021000501721.000000,70004.000000,1.000000,1.000000,52.000000,1.000000,22.000000,1.000000,1959.000000,0.000000,0.000000,0.000000,1.000000,5.000000,50.000000,0.000000,0.000000,0.000000,0.000000,1.000000,10.000000,3820.000000,0.000000,0.000000,41000.000000,0.000000
50%,2021.000000,202101.000000,104849.500000,2021000599900.500000,73.000000,2021001048491.000000,371506.000000,1.000000,2.000000,75.000000,2.000000,42.000000,4.000000,1979.000000,1.000000,1.000000,1.000000,1.000000,6.000000,65.000000,0.000000,0.000000,0.000000,0.000000,1.000000,10.000000,7340.000000,3291.000000,0.000000,85300.000000,20000.000000
75%,2021.000000,202101.000000,145638.750000,2021000896807.000000,119.000000,2021001456381.000000,651506.000000,1.000000,3.000000,124.000000,2.000000,62.000000,6.000000,1999.000000,1.000000,1.000000,1.000000,1.000000,10.000000,101.000000,19.000000,1901.000000,0.000000,0.000000,3.000000,30.000000,9920.000000,7770.000000,52.000000,161400.000000,95000.000000
max,2021.000000,202101.000000,186204.000000,2021010160574.000000,1616.000000,2021001862031.000000,1130006.000000,5.000000,20.000000,1617.000000,2.000000,94.000000,6.000000,2021.000000,3.000000,2.000000,2.000000,2.000000,11.000000,116.000000,64.000000,6403.000000,64.000000,6403.000000,3.000000,30.000000,9920.000000,9920.000000,52.000000,9999999.000000,999999.000000


In [7]:
df["FTOTINC"].value_counts()

9999999.0    25777
0.0           7322
30000.0       3617
50000.0       3614
60000.0       3460
             ...  
68290.0          1
25960.0          1
140960.0         1
13060.0          1
79550.0          1
Name: FTOTINC, Length: 13311, dtype: int64

In [8]:
df["INCWAGE"].value_counts()

0.0         165669
999999.0     73360
30000.0       6411
50000.0       5784
40000.0       5709
             ...  
297000.0         1
374000.0         1
353000.0         1
393000.0         1
352000.0         1
Name: INCWAGE, Length: 937, dtype: int64

In [9]:
df_nan = df.copy()
df_nan["FTOTINC"] = df["FTOTINC"].replace(9999999, np.nan)
df_nan["INCWAGE"] = df["INCWAGE"].replace(999999, np.nan)

df_nan.max()

<ipython-input-9-26f5ad44bca3>:5: FutureWarning: The default value of numeric_only in DataFrame.max is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_nan.max()


YEAR          2.021000e+03
SAMPLE        2.021010e+05
SERIAL        1.862040e+05
CBSERIAL      2.021010e+12
HHWT          1.616000e+03
CLUSTER       2.021002e+12
STRATA        1.130006e+06
GQ            5.000000e+00
PERNUM        2.000000e+01
PERWT         1.617000e+03
SEX           2.000000e+00
AGE           9.400000e+01
MARST         6.000000e+00
BIRTHYR       2.021000e+03
MARRNO        3.000000e+00
DIVINYR       2.000000e+00
WIDINYR       2.000000e+00
SCHOOL        2.000000e+00
EDUC          1.100000e+01
EDUCD         1.160000e+02
DEGFIELD      6.400000e+01
DEGFIELDD     6.403000e+03
DEGFIELD2     6.400000e+01
DEGFIELD2D    6.403000e+03
EMPSTAT       3.000000e+00
EMPSTATD      3.000000e+01
OCC2010       9.920000e+03
IND           9.920000e+03
WKSWORK1      5.200000e+01
FTOTINC       2.508000e+06
INCWAGE       6.270000e+05
dtype: float64

### Imputation of missing data 
There are several techniques we may impute the missing data, and we will examine the bias-variance tradeoff using the [MLxtend](http://rasbt.github.io/mlxtend/) Library. 


*   Substitution by mean
*   Substitution by median
*   Model Imputation
*   Maximum Likelihood Imputation
*   Multiple Imputation 




#### Substitution by mean 
I am treating the NaN data as the target variable and doing simple imputation. 

In [10]:
df_nan.isnull().sum()

YEAR              0
SAMPLE            0
SERIAL            0
CBSERIAL          0
HHWT              1
CLUSTER           1
STRATA            1
GQ                1
PERNUM            1
PERWT             1
SEX               1
AGE               1
MARST             1
BIRTHYR           1
MARRNO            1
DIVINYR           1
WIDINYR           1
SCHOOL            1
EDUC              1
EDUCD             1
DEGFIELD          1
DEGFIELDD         1
DEGFIELD2         1
DEGFIELD2D        1
EMPSTAT           1
EMPSTATD          1
OCC2010           1
IND               1
INDNAICS          1
WKSWORK1          1
FTOTINC       25778
INCWAGE       73361
dtype: int64

In [11]:
from sklearn.impute import SimpleImputer
train_mean = df_nan[['BIRTHYR','FTOTINC','INCWAGE']].copy()
mean_imputer = SimpleImputer(strategy='mean')
train_mean.iloc[:,:] = mean_imputer.fit_transform(train_mean)
train_mean.isnull().sum()
df_mean = df_nan.fillna(train_mean)
df_mean.isnull().sum()

YEAR          0
SAMPLE        0
SERIAL        0
CBSERIAL      0
HHWT          1
CLUSTER       1
STRATA        1
GQ            1
PERNUM        1
PERWT         1
SEX           1
AGE           1
MARST         1
BIRTHYR       0
MARRNO        1
DIVINYR       1
WIDINYR       1
SCHOOL        1
EDUC          1
EDUCD         1
DEGFIELD      1
DEGFIELDD     1
DEGFIELD2     1
DEGFIELD2D    1
EMPSTAT       1
EMPSTATD      1
OCC2010       1
IND           1
INDNAICS      1
WKSWORK1      1
FTOTINC       0
INCWAGE       0
dtype: int64

#### Substitution by median

In [12]:
from sklearn.impute import SimpleImputer
train_median = df_nan[['BIRTHYR','FTOTINC','INCWAGE']].copy()
median_imputer = SimpleImputer(strategy='median')
train_median.iloc[:,:] = median_imputer.fit_transform(train_median)
train_median.isnull().sum()
df_median = df_nan.fillna(train_median)
df_median.isnull().sum()

YEAR          0
SAMPLE        0
SERIAL        0
CBSERIAL      0
HHWT          1
CLUSTER       1
STRATA        1
GQ            1
PERNUM        1
PERWT         1
SEX           1
AGE           1
MARST         1
BIRTHYR       0
MARRNO        1
DIVINYR       1
WIDINYR       1
SCHOOL        1
EDUC          1
EDUCD         1
DEGFIELD      1
DEGFIELDD     1
DEGFIELD2     1
DEGFIELD2D    1
EMPSTAT       1
EMPSTATD      1
OCC2010       1
IND           1
INDNAICS      1
WKSWORK1      1
FTOTINC       0
INCWAGE       0
dtype: int64

#### Iterative Imputer

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

itimp_FTOTINC = pd.DataFrame(np.array(df_nan['FTOTINC']).reshape(-1,1))
imp_mean = IterativeImputer(random_state=0)
imp_mean.fit_transform(itimp_FTOTINC)
itimp_FTOTINC.isnull().sum()


0    25778
dtype: int64

In [14]:
itimp_INCWAGE = pd.DataFrame(np.array(df_nan['INCWAGE']).reshape(-1,1))
imp_mean.fit_transform(itimp_INCWAGE)
itimp_INCWAGE.isnull().sum()

0    73361
dtype: int64

#### K-Nearest Neighbors Imputer 




In [15]:
from sklearn.impute import KNNImputer
KNNImp_FTOTINC = pd.DataFrame(np.array(df_nan['FTOTINC']).reshape(-1,1))
KNNImp_INCWAGE = pd.DataFrame(np.array(df_nan['INCWAGE']).reshape(-1,1))
imputer = KNNImputer(n_neighbors=5)
imputer.fit_transform(KNNImp_FTOTINC)
imputer = KNNImputer(n_neighbors=5)
imputer.fit_transform(KNNImp_INCWAGE)

array([[    0.        ],
       [    0.        ],
       [    0.        ],
       ...,
       [    0.        ],
       [20000.        ],
       [33447.46803356]])

## Part 2: Creating the database & SQL 

In [16]:
conn = sqlite3.connect('censusData.db')
df_median.to_sql('censusDatabase', con = conn, if_exists='replace', index=False)
pd.read_sql('SELECT * FROM censusDatabase WHERE AGE > "40"', con = conn).head()

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,STRATA,GQ,PERNUM,PERWT,...,DEGFIELD2,DEGFIELD2D,EMPSTAT,EMPSTATD,OCC2010,IND,INDNAICS,WKSWORK1,FTOTINC,INCWAGE
0,2021,202101,1,2021010000026,13.0,2.021000e+12,80001.0,3.0,1.0,13.0,...,0.0,0.0,3.0,30.0,9920.0,0.0,0,0.0,80000.0,0.0
1,2021,202101,2,2021010000031,51.0,2.021000e+12,80001.0,3.0,1.0,51.0,...,0.0,0.0,3.0,30.0,9920.0,0.0,0,0.0,80000.0,0.0
2,2021,202101,3,2021010000063,17.0,2.021000e+12,120001.0,3.0,1.0,17.0,...,0.0,0.0,3.0,30.0,9920.0,0.0,0,0.0,80000.0,0.0
3,2021,202101,5,2021010000100,15.0,2.021000e+12,50001.0,3.0,1.0,15.0,...,0.0,0.0,3.0,30.0,9920.0,0.0,0,0.0,80000.0,0.0
4,2021,202101,9,2021010000401,71.0,2.021000e+12,120001.0,3.0,1.0,71.0,...,0.0,0.0,3.0,30.0,9920.0,0.0,0,0.0,80000.0,0.0


References: 

**Data Set**

Steven Ruggles, Sarah Flood, Matthew Sobek, Danika Brockman, Grace Cooper,  Stephanie Richards, and Megan Schouweiler. IPUMS USA: Version 13.0 [dataset]. Minneapolis, MN: IPUMS, 2023. https://doi.org/10.18128/D010.V13.0